In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import os
from glob import glob

### 백터저장소 로드

In [11]:
### 백터저장소 로드


from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    chunk_size=1,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",       
)

print(f"벡터 저장소에 저장된 문서 수 : {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수 : 12


In [12]:
#다중 메시지 전송

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",    
    temperature=0.0,
    max_tokens=100,
)

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

prompt = ChatPromptTemplate.from_messages(messages)    
print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [13]:
print(prompt.input_variables)

['query']


In [14]:
prompt_text = prompt.format_prompt(query="테슬라의 창업자는 누구인가요?")
print(prompt_text)

messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content='테슬라의 창업자는 누구인가요?')]


In [15]:
response = llm.invoke(prompt_text)
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사를 이끌어왔습니다. 따라서


In [16]:
#LCEL 체인 구성

chain = prompt | llm 

print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001BB75FD3190>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001BB75FF4950>, root_client=<openai.OpenAI object at 0x000001BB75FD3A90>, root_async_client=<openai.AsyncOpenAI object at 0x000001BB75FE7C50>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [17]:
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [18]:
response = chain.invoke({"query": "테슬라의 창업자는 누구인가요?"})
print(response.content)

테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)입니다. 이 두 사람은 2003년에 테슬라 모터스를 설립했습니다. 이후 엘론 머스크(Elon Musk)가 2004년에 투자자로 참여하면서 회사의 주요 인물 중 하나가 되었고, 이후 CEO로 취임하여 테슬라의 성장에 큰 기여를 하였습니다


prompt + llm + Output Parser

In [19]:
response

AIMessage(content='테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)입니다. 이 두 사람은 2003년에 테슬라 모터스를 설립했습니다. 이후 엘론 머스크(Elon Musk)가 2004년에 투자자로 참여하면서 회사의 주요 인물 중 하나가 되었고, 이후 CEO로 취임하여 테슬라의 성장에 큰 기여를 하였습니다', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 28, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'length', 'logprobs': None}, id='run-9b19f339-125e-4f00-9553-942c98c5ea2a-0', usage_metadata={'input_tokens': 28, 'output_tokens': 100, 'total_tokens': 128})

StrOutputParser

In [20]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

'테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)입니다. 이 두 사람은 2003년에 테슬라 모터스를 설립했습니다. 이후 엘론 머스크(Elon Musk)가 2004년에 투자자로 참여하면서 회사의 주요 인물 중 하나가 되었고, 이후 CEO로 취임하여 테슬라의 성장에 큰 기여를 하였습니다'

In [21]:
#체인 걸기

str_chain = prompt | llm | output_parser

query = "리비안의 설립 연도는 언제인가요?"
str_response = str_chain.invoke(query)
print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 기업으로, 특히 전기 픽업트럭과 SUV 모델에 주력하고 있습니다.


JSON 출력 : JsonOutputParser

In [22]:
##json

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_response = chain.invoke("테슬라 창업자는 누구인가요? json 형식으로 답해주세요.")

print(json_response)
print('====================')

json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": "엘론 머스크",\n  "설명": "테슬라는 2003년에 마틴 에버하드와 마크 타페닝에 의해 설립되었지만, 엘론 머스크가 2004년에 투자하고 CEO로 취임하면서 회사의 주요 인물로 알려지게 되었습니다."\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 34, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run-a20e1f0d-0cba-480f-afc6-0329f0a31f89-0' usage_metadata={'input_tokens': 34, 'output_tokens': 80, 'total_tokens': 114}
{'창업자': '엘론 머스크', '설명': '테슬라는 2003년에 마틴 에버하드와 마크 타페닝에 의해 설립되었지만, 엘론 머스크가 2004년에 투자하고 CEO로 취임하면서 회사의 주요 인물로 알려지게 되었습니다.'}


PydanticOutputParser

In [23]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

#BaseModel기반으로 우리만의 도메인 객체 정의(생성)
class Person(BaseModel):
    """Information about a person."""
    
    name: str = Field(description="The name of the person")
    title: str = Field(description="The title or position of the person")
# llm에서 받아온 객체 person 객체로 매핑   
pydantic_parser = PydanticOutputParser(pydantic_object=Person)
                    #PydanticOutputParser를 사용하여 구조화된 객체 출력 
                    
# print('Pydantic output Parser prompt :')
# print(pydantic_parser.get_format_instructions()) #get_format_instructions()함수가 지시해줌. 
# #아직은 비교가 불가능 

# print('====================')
#프롬프트 만들기
prompt = ChatPromptTemplate.from_messages([
    ("system", "Anser the user query Wrap the output in 'json' format. tags\n{format_instructions}"),
    #json 형식으로 답변 받아서 아래 객체로 매핑
    ("user", "{query}"), #human
]).partial(format_instructions=pydantic_parser.get_format_instructions())


In [24]:
#pydantic_parser 이 체인을 걸어서 원하는 형태로 출력되도록

person_chain = prompt | llm | pydantic_parser

#pydantic_parser 
# 챗봇은 그냥 스트링으로 받아와도 됨. 
# 파이선 객체로 만들어서 사용해야할 때 
# 결과값을 가지고 도메인 매핑을 해서 출력되도록 해줌. 
response = person_chain.invoke({"query": "테슬라의 창업자는 누구인가요?"})  
print(response)
response

name='엘론 머스크' title='CEO 및 공동 창립자'


Person(name='엘론 머스크', title='CEO 및 공동 창립자')

In [25]:
import time

for chunk in llm.stream('테슬라 창업자는 누구인가요?'):
    print(chunk.content, end='', flush=True)
    time.sleep(0.1) 


테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 방향성을 크게 변화시켰습니다. 이후 그는 테슬라의 가장 유명한 얼굴이

Batch

In [26]:
#batch
questions = [
    "테슬라 창업자는 누구인가요?",
    "리비안의 설립년도는 언제인가요?"
]

response = llm.batch(questions)

for response in response:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다. 이후 그는 테슬라의 가장 잘 알려진 얼굴이

================================== Ai Message ==================================

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.



Runnable

In [30]:
retriever = vectorstore.as_retriever(
    search_kwarks={'k':1}
)

query =  "테슬라 창업자는 누구인가요?"
retrived_docs = retriever.invoke(query)
print(f"검색된 문서 수: {len(retrived_docs)}")
retieved_docs_text = "\n".join([doc.page_content for doc in retrived_docs])
pprint(retieved_docs_text)

검색된 문서 수: 4
('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010\n'
 '명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.\n'
 '명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매

In [31]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

#RunnablePassthrough: 입력을 그대로 전달하거나 추가 정보와 함께 전달
# RunnableParallel : 여러 작업을 동시에 병렬 실행
runnable = RunnableParallel(
    question = RunnablePassthrough(),
    
)

runnable.invoke({"question": '테슬라 창업자는 누구?'})

{'question': {'question': '테슬라 창업자는 누구?'}}

In [32]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question = RunnablePassthrough(),
    word_count = RunnableLambda(count_num_words)
)

runnable.invoke('테슬라 창업자는 누구?')

{'question': '테슬라 창업자는 누구?', 'word_count': 3}

In [33]:
#prompt 템플릿 생성
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based only on the following context.
Do not use any external knowledge.
If the answer is not in the context, say "I don't know."

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.pretty_print()



================================ Human Message =================================


Answer the question based only on the following context.
Do not use any external knowledge.
If the answer is not in the context, say "I don't know."

[Context]
{context}

[Question]
{question}

[Answer]



In [34]:
#벡터 검색기

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 2}, # => 디비로부터 가져온context임. 위에 {context}로 감. 
)

#문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

#체인구성
retriever_chain = retriever | format_docs
            #결과값을 format_docs 체인을 걸어서 구성

#체인 실행
response = retriever_chain.invoke("테슬라 창업자 누구?")
pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010\n'
 '\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010')


RAG체인과 연결

In [40]:
#RAG체인과 연결

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
)

#LLM체인 생성
    #retriever_chain : 포맷함수로 한 문장으로 만들어줌.  RunnablePassthrough: 사용자 입력한거 그대로 쓰겟다
rag_chain = (
    {"context": retriever_chain, "questions": RunnablePassthrough}
    | prompt
    | llm
    | StrOutputParser()
)

#RAG체인실행

response = rag_chain.invoke("테슬라 창업자는 누구?")
print(response)

ValidationError: 2 validation errors for RunnablePassthrough
func
  테슬라 창업자는 누구? is not callable (type=type_error.callable; value=테슬라 창업자는 누구?)
func
  테슬라 창업자는 누구? is not callable (type=type_error.callable; value=테슬라 창업자는 누구?)

In [41]:
# 필요한 import 추가
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
)

# RAG 체인 생성 (수정됨)
rag_chain = (
    {"context": retriever_chain, "question": RunnablePassthrough()}  # () 추가, question으로 변경
    | prompt
    | llm
    | StrOutputParser()
)

# RAG 체인 실행
response = rag_chain.invoke("테슬라 창업자는 누구?")
print(response)

테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.


In [43]:

import gradio as gr
def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

demo = gr.ChatInterface(fn=answer_invoke, title="miny bot")

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\KOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-e_qH7Re7-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
